In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer
from lightning.pytorch import seed_everything

from embedder import TextEmbedder
from training_tools import train_model, test_model

In [2]:
DETERMINISTIC = True
BATCH_SIZE = 8

In [3]:
if DETERMINISTIC:
    seed_everything(42)

Global seed set to 42


In [4]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [5]:
dataset = load_dataset("sst2").with_format('torch')
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
model = TextEmbedder('xlm-roberta-base')

Found cached dataset sst2 (/home/proto/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5)


  0%|          | 0/3 [00:00<?, ?it/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

In [6]:
train_ds = dataset['train'].map(lambda x: tokenizer(x['sentence'], truncation=True, padding='max_length'), batched=True)
val_ds = dataset['validation'].map(lambda x: tokenizer(x['sentence'], truncation=True, padding='max_length'), batched=True)
test_ds = dataset['test'].map(lambda x: tokenizer(x['sentence'], truncation=True, padding='max_length'), batched=True)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [7]:
from torch.utils.data import DataLoader

dl = DataLoader(train_ds, batch_size=64)

for batch in dl:
    print(batch.keys())
    break

dict_keys(['idx', 'sentence', 'label', 'input_ids', 'attention_mask'])


In [8]:
tokenizer = None
trainer = train_model(model, train_ds, val_ds, batch_size=BATCH_SIZE, max_epochs=5, deterministic=DETERMINISTIC)

Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/proto/repos/crisis-detector/.conda/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory /home/proto/repos/crisis-detector/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type                                | Params
----------------------------------------------------------------
0 | loss_fn | CrossEntropyLoss                    | 0     
1 | f1      | MulticlassF1Score                   | 0     
2 | acc     | MulticlassAccuracy                  | 0     
3 | prec    | MulticlassPrecision                 | 0     
4 | rec     | MulticlassRecall                    | 0     
5 | model   | XLMRobertaForSequenc

Sanity Checking: 0it [00:00, ?it/s]

/home/proto/repos/crisis-detector/.conda/lib/python3.10/site-packages/lightning/pytorch/utilities/data.py:76: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 8. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


Training: 0it [00:00, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 734.00 MiB (GPU 0; 10.00 GiB total capacity; 6.30 GiB already allocated; 93.00 MiB free; 8.68 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
test_model(test_ds, trainer=trainer, batch_size=BATCH_SIZE, deterministic=DETERMINISTIC)